In [32]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [36]:
#chrome_driver_path = r"C:\Users\popth\Downloads\chrome-win64\chrome-win64"
"""options=Options()
options.add_argument(r"--user-data-dir=C:\\Users\\popth\\AppData\\Local\\Google\\Chrome\\User Data")
options.add_argument(r'--profile-directory=Profile 4')
options.add_argument(r"--remote-debugging-port=9222")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

driver=webdriver.Chrome(service=Service(executable_path=chrome_driver_path),options=options)"""


driver=webdriver.Chrome()
driver.maximize_window()

# explicit wait
wait=WebDriverWait(driver,20)

# defining a function if the page is fully loaded or not
def wait_for_page_to_load(driver,wait):
    page_title=driver.title
    try:
        wait.until(
            lambda d: d.execute_script("return document.readyState")=="complete"
        )
    except:
        print(f"The page {page_title} did not get fully loaded within the given duration.")
    else:
        print(f"The page {page_title} is fully loaded.")
        
url="https://www.finance.yahoo.com"
driver.get(url)
wait_for_page_to_load(driver,wait)


# hovering on Markets menu
actions=ActionChains(driver)
markets_menu=wait.until(
    EC.presence_of_element_located((By.XPATH,'/html[1]/body[1]/div[2]/header[1]/div[1]/div[1]/div[1]/div[4]/div[1]/div[1]/ul[1]/li[3]/a[1]/span[1]'))
)
actions.move_to_element(markets_menu).perform()


# click on Trending tickers under Markets
trending_tickers=wait.until(
    #EC.element_to_be_clickable((By.XPATH,'//*[@id="ybar-inner-wrap"]/div[2]/div/div[1]/div[1]/div/div/div[1]/section[4]/ul/li[3]/ul/li[5]/a'))
    EC.element_to_be_clickable((By.LINK_TEXT,"Trending Tickers"))
)
trending_tickers.click()

# click on Most Active
most_active=wait.until(
    EC.element_to_be_clickable((By.XPATH,'/html[1]/body[1]/div[2]/main[1]/section[1]/section[1]/section[1]/article[1]/section[1]/div[1]/nav[1]/ul[1]/li[1]/a[1]/span[1]'))
)
most_active.click()


# scraping the data
data=[]    # the extracted data will be stored in this list
while True:
    # scraping the data from the table
    wait.until(
        EC.presence_of_element_located((By.TAG_NAME,"table"))
    )
    rows=driver.find_elements(By.CSS_SELECTOR,"table tbody tr")
    for row in rows:
        values=row.find_elements(By.TAG_NAME,"td")
        stock={
            "name":values[1].text,
            "symbol":values[0].text,
            "price":values[3].text,
            "change":values[4].text,
            "volume":values[6].text,
            "market_cap":values[8].text,
            "pe_ratio":values[9].text
        }
        data.append(stock)

    # click next
    try:
        next_button=wait.until(
            EC.element_to_be_clickable((By.XPATH,'//*[@id="nimbus-app"]/section/section/section/article/section[1]/div/div[3]/div[3]/button[3]'))
        )
    except:
        print("The \"next button\" is not clickable. We have navigated through all the pages.")
        break
    else:
        next_button.click()
        time.sleep(1)
#driver.quit()

The page Yahoo Finance - Stock Market Live, Quotes, Business & Finance News is fully loaded.
The "next button" is not clickable. We have navigated through all the pages.


## Its time to clean the data now. Let's do that

In [165]:
import pandas as pd
import numpy as np
# converting the data into a pandas dataframe
stocks_df=(pd.DataFrame(data)
            .apply(lambda col: col.str.strip() if col.dtype=="object" else col)
            .assign(price=lambda df_: pd.to_numeric(df_.price),
                   change=lambda df_: df_.change.str.replace("+",""),
                    volume=lambda df_: pd.to_numeric(df_.volume.str.replace("M","")),
                    pe_ratio=lambda df_:pd.to_numeric(df_
                                         .pe_ratio.replace("--",np.nan)
                                         .str.replace(",","")
                                        ),
                    market_cap=lambda df_: df_.market_cap.apply(lambda val: float(val.replace("B","")) if "B" in val else float(val.replace("T",""))*1000)
                   )
            .rename(columns={
               "price":"price_usd",
                "volume":"volume_M",
                "market_cap":"market_cap_B"
            })
           .assign(change=lambda df_:pd.to_numeric(df_.change))
          )

#stocks_df.pe_ratio.str.extract(r"([^0-9.])",expand=False).unique()   # to check if there is any other character in each of the elements other than 0-9 and "."
stocks_df

,name,symbol,price_usd,change,volume_M,market_cap_B,pe_ratio
0,Berkshire Hathaway Inc.,LCID,2.38,-0.15,79.986,7.256,NaN
1,NVIDIA Corporation,NVDA,113.74,-0.76,53.522,2775.000,38.69
2,"Skechers U.S.A., Inc.",SKX,61.60,12.23,47.695,9.270,14.77
3,Palantir Technologies Inc.,PLTR,124.01,-0.27,45.115,292.623,652.68
4,"Tesla, Inc.",TSLA,276.56,-10.65,41.116,890.789,158.03
...,...,...,...,...,...,...,...
76,"Tyson Foods, Inc.",TSN,55.33,-5.47,5.227,19.706,18.69
77,Microsoft Corporation,MSFT,436.51,1.23,5.198,3244.000,33.76
78,Wells Fargo & Company,WFC,74.58,0.78,5.153,242.713,13.41
79,"The Trade Desk, Inc.",TTD,56.13,2.04,5.102,27.592,71.96


In [166]:
! pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: C:\Users\popth\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [168]:
stocks_df.to_excel("yahoo-data.xlsx",index=False)